# 初期化フェーズ

In [ ]:
import os

# カレントディレクトリをmagentaに
os.chdir(r"C:\Users\arkw\GitHub\magenta")
print(os.getcwd())

In [ ]:
SCRIPT_PATH = 'magenta\models\music_vae\music_vae_generate.py' # 実行したいスクリプトのファイル名
MODEL_CONFIG = 'hierdec-mel_16bar' # コンフィグ
CHECKPOINT_FILE = 'checkpoints\hierdec-mel_16bar.tar' # チェックポイントファイルの場所
NUM_OUTPUTS = 5 # 初期個体の数
OUTPUT_DIR = 'tmp\music_vae\generated' # 基本となる出力場所
TEMPERATURE = str(0.001)

iec_dir = 'iec1125_LoA' # 今回のMIDIやベクトルを保存する場所
generation = 1 # 世代
midi_dir = os.path.join(OUTPUT_DIR, iec_dir, f"gen_{generation:03d}")
vectors_dir = os.path.join(OUTPUT_DIR, iec_dir, f"gen_{generation:03d}")

input_attribute_1 = 'checkpoints/attributes/valence.npy'
input_attribute_2 = 'checkpoints/attributes/arousal.npy'
magnitude_1 = str(0)
magnitude_2 = str(-1)

sample_mean = "checkpoints\VA_mean_std_241122_ratio0.1\arousal_score_high_mean_array.npy"
sample_std = "checkpoints\VA_mean_std_241122_ratio0.1\arousal_score_high_std_array.npy"

In [ ]:
# 初期個体が気に入らなければリセット

import os

# ディレクトリ内の全ファイルをリストアップ
for filename in os.listdir(midi_dir):
    # ファイルのフルパスを作成
    file_path = os.path.join(midi_dir, filename)
    # 拡張子が .mid のファイルを削除
    if filename.endswith('.mid'):
        os.remove(file_path)
        print(f"Deleted: {file_path}")

In [ ]:
import subprocess
import os

# スクリプトに渡したい引数のリスト
args = [
    '--config='+MODEL_CONFIG,
    '--checkpoint_file='+CHECKPOINT_FILE,
    '--mode=sample_withattr',
    '--num_outputs='+str(NUM_OUTPUTS),
    '--output_dir='+midi_dir,
    '--temperature='+str(TEMPERATURE),
    '--input_attribute_1='+input_attribute_1,
    '--input_attribute_2='+input_attribute_2,
    '--magnitude_1='+str(magnitude_1),
    '--magnitude_2='+str(magnitude_2),
    '--savez=True'
]

# subprocess.runを使用してスクリプトを実行
# 引数はリストの形で渡す
result = subprocess.run(['python', SCRIPT_PATH] + args, cwd=os.getcwd(), capture_output=True, text=True)

# 実行結果の出力
# print('stdout:', result.stdout)
print('stderr:', result.stderr)

In [ ]:
# MIDI再生の関数たち

import numpy as np
import os
# from music21 import *
from IPython.display import Audio, display
from midi2audio import FluidSynth
import pretty_midi
import matplotlib.pyplot as plt

def play_midi(file_path):
    """指定されたパスのMIDIファイルを再生する関数"""
    try:
        # print(file_path)
        # score = converter.parse(file_path)
        # score.show()

        midi = pretty_midi.PrettyMIDI(file_path)
        # pianoroll = midi.get_piano_roll()
        if midi.instruments:
            notes = midi.instruments[0].notes
        # print(pianoroll.shape)
        # ピアノロールの表示設定を調整
        plt.figure(figsize=(9, 3))  # 図のサイズを設定（幅、高さ）

        # plt.imshow(pianoroll, 
        #            aspect=6,
        #            cmap='binary')  # aspectで縦に広げる
        
        # # 値が80の位置を見つけて黒い枠を描画
        # rows, cols = np.where(pianoroll == 80)
        # for r, c in zip(rows, cols):
        #     plt.gca().add_patch(plt.Rectangle((c-0.5, r-0.5), 0, 1, ec="red", lw=0.05, fill=False))

        for note in notes:
            plt.gca().add_patch(plt.Rectangle(
                (midi.time_to_tick(note.start)/4.4, note.pitch), 
                 midi.time_to_tick(note.get_duration()/4.4), 1, fc="cyan", ec="k"))

        plt.ylabel('Pitch')
        plt.xlabel('Time')
        plt.title(file_path)
        # plt.tight_layout()  # レイアウトを自動調整
        plt.gca().invert_yaxis() # y軸反転
        plt.xlim(0, 3200)
        plt.ylim(24, 96)
        plt.xticks(np.arange(0, 3200, 200))
        plt.yticks(np.arange(24, 96, 12))
        plt.grid(alpha=0.5)
        plt.show()

        fs = FluidSynth(sound_font="TimGM6mb.sf2")
        fs.midi_to_audio(file_path, 'output.wav')
        display(Audio('output.wav', autoplay=False))
        
    except Exception as e:
        print(f"Failed to play {file_path}: {e}")

def play_folder(folder_path):
    """フォルダ内のすべてのMIDIファイルを再生する"""
    # フォルダ内のファイルをリストアップ
    files = [f for f in os.listdir(folder_path) if f.endswith('.mid')]
    # 各ファイルを再生
    for file in files:
        file_path = os.path.join(folder_path, file)
        play_midi(file_path)

In [ ]:
# フォルダ内のMIDIを再生
# midi_dir = "tmp/music_vae/generated/iec1125_HiA/gen_001"
play_folder(midi_dir)

In [ ]:
print(os.getcwd())

In [ ]:
import magenta.models.music_vae.save_latent_vectors

# magenta.models.music_vae.save_latent_vectors.save_z(midi_dir, vectors_dir, MODEL_CONFIG, CHECKPOINT_FILE)
magenta.models.music_vae.save_latent_vectors.save_latent_vectors(midi_dir, vectors_dir, MODEL_CONFIG, CHECKPOINT_FILE)

# 繰り返しフェーズの準備

In [ ]:
# 単峰性正規分布交叉（UNDX）

import numpy as np

def gram_schmidt(A):
    Q = np.zeros(A.shape)
    for i in range(A.shape[1]):
        # i列目のベクトルを取得
        q = A[:, i]
        for j in range(i):
            # qから既に計算されたベクトルの影響を取り除く
            q = q - np.dot(Q[:, j], A[:, i]) * Q[:, j]
        # qを正規化して、Qのi列目に設定
        Q[:, i] = q / np.linalg.norm(q)
    return Q

def distance_point_to_line(point1, point2, point3):
    """
    点1と点2を通る直線と点3の距離を計算する
    
    入力:
        point1: ndarray, shape=(n,), 直線上の点1
        point2: ndarray, shape=(n,), 直線上の点2 
        point3: ndarray, shape=(n,), 距離を求める点
        
    出力:
        距離: float
    """
    # 直線の方向ベクトルを計算
    line_vec = point2 - point1
    # 点3から直線への射影ベクトルを計算
    proj_vec = np.dot((point3 - point1), line_vec) / np.linalg.norm(line_vec)**2 * line_vec
    # 点3から直線への垂線ベクトルを計算
    perp_vec = point3 - point1 - proj_vec
    # 距離を計算
    distance = np.linalg.norm(perp_vec)
    
    return distance

def undx(parent1, parent2, parent3, sigma_xi=0.5, sigma_eta=0.35):
    """
    単峰性正規分布交叉の実装
    個体群から3つの親、すなわちparent1, parent2, parent3が選ばれます。
    mをparent1, parent2の中点、dをparent1, parent2の差ベクトルとします。
    dで表される方向は一次探索方向と呼ばれます。このベクトルに直交する部分空間は二次探索方向と呼ばれます。
    Dはparent1, parent2を結ぶ線と3番目の親parent3との間の距離です。
    ベクトルe_iは二次探索方向を広げる正規直交基底であり、dに垂直かつ線形独立な単位ベクトルです。
    """
    # 親個体の次元を取得
    n = parent1.size
    # 中心点 m を計算
    m = (parent1 + parent2) / 2.0
    # 親個体間の距離ベクトル d を計算
    d = parent2 - parent1
    # parent3と線の間の距離Dを計算
    D = distance_point_to_line(parent1, parent2, parent3)
    
    # dに垂直なベクトルを生成
    E = np.eye(n)
    d_norm = d / np.linalg.norm(d)
    E[:, 0] = d_norm
    Q = gram_schmidt(E)
    e = Q[:, 1:]  # dに垂直なベクトルを選択

    # 正規分布から乱数を生成
    xi = np.random.normal(0, sigma_xi)
    eta = np.random.normal(0, sigma_eta/np.sqrt(n), n-1)

    # 子個体を生成
    child = m + xi * d + D * np.dot(e, eta)
    return child

In [ ]:
# 遺伝的アルゴリズムの関数・改

import numpy as np

# 選択関数（ルーレット選択）
def roulette_select(arrays, scores):
    probability_distribution = scores / scores.sum()
    parents_indices = np.random.choice(
        len(arrays), size=len(arrays), p=probability_distribution, replace=False)
    selected_parents = []
    for i in range(len(arrays)):
        selected_parents.append(arrays[parents_indices[i]])
    
    return selected_parents

# 交叉関数（一点交叉）使わない
def crossover(parent1, parent2):
    crossover_point = np.random.randint(parent1.size)  # 交叉点をランダムに選択
    child1 = np.hstack((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.hstack((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

# 突然変異関数
def mutate(array, mutation_rate=0.01):
    mutation_mask = np.random.rand(array.size) < mutation_rate
    array[mutation_mask] = np.random.normal(size=np.sum(mutation_mask))
    return array

# 遺伝的アルゴリズムの実行
def evolutionary_computation(arrays, scores, mutation_rate=0.01):
    # 親の選択
    parents = roulette_select(arrays, scores)
    
    # UNDX
    child = undx(parents[0], parents[1], parents[2])
    
    # # 突然変異
    # child1 = mutate(child1, mutation_rate)
    # child2 = mutate(child2, mutation_rate)
    
    return child

In [ ]:
# 単峰性正規分布交叉（UNDX）（旧）

import numpy as np

def gram_schmidt(A):
    Q = np.zeros(A.shape)
    for i in range(A.shape[1]):
        # i列目のベクトルを取得
        q = A[:, i]
        for j in range(i):
            # qから既に計算されたベクトルの影響を取り除く
            q = q - np.dot(Q[:, j], A[:, i]) * Q[:, j]
        # qを正規化して、Qのi列目に設定
        Q[:, i] = q / np.linalg.norm(q)
    return Q

def distance_point_to_line(point1, point2, point3):
    """
    点1と点2を通る直線と点3の距離を計算する
    
    入力:
        point1: ndarray, shape=(1, n), 直線上の点1
        point2: ndarray, shape=(1, n), 直線上の点2 
        point3: ndarray, shape=(1, n), 距離を求める点
        
    出力:
        距離: float
    """
    # 直線の方向ベクトルを計算
    line_vec = point2 - point1
    # 点3から直線への射影ベクトルを計算
    proj_vec = np.dot((point3 - point1), line_vec.transpose()) / np.linalg.norm(line_vec)**2 * line_vec
    # 点3から直線への垂線ベクトルを計算
    perp_vec = point3 - point1 - proj_vec
    # 距離を計算
    distance = np.linalg.norm(perp_vec)
    
    return distance

def undx(parent1, parent2, parent3, sigma_xi=0.5, sigma_eta=0.35):
    """
    単峰性正規分布交叉の実装
    個体群から3つの親、すなわちparent1, parent2, parent3が選ばれます。
    mをparent1, parent2の中点、dをparent1, parent2の差ベクトルとします。
    dで表される方向は一次探索方向と呼ばれます。このベクトルに直交する部分空間は二次探索方向と呼ばれます。
    Dはparent1, parent2を結ぶ線と3番目の親parent3との間の距離です。
    ベクトルe_iは二次探索方向を広げる正規直交基底であり、dに垂直かつ線形独立な単位ベクトルです。
    """
    # 親個体の次元を取得
    n = parent1.shape[1]
    # 中心点 m を計算
    m = (parent1 + parent2) / 2.0
    # 親個体間の距離ベクトル d を計算
    d = parent2 - parent1
    # parent3と線の間の距離Dを計算
    D = distance_point_to_line(parent1, parent2, parent3)
    
    # dに垂直なベクトルを生成
    E = np.eye(n)
    d_norm = d / np.linalg.norm(d)
    E[:, 0] = d_norm
    Q = gram_schmidt(E)
    e = Q[:, 1:]  # dに垂直なベクトルを選択

    # 正規分布から乱数を生成
    xi = np.random.normal(0, sigma_xi)
    eta = np.random.normal(0, sigma_eta/np.sqrt(n), n-1)

    # 子個体を生成
    child = m + xi * d + D * np.dot(e, eta)
    return child

In [ ]:
# 遺伝的アルゴリズムの関数（旧）

import numpy as np

# 選択関数（ルーレット選択）
def roulette_select(arrays, scores):
    probability_distribution = scores / scores.sum()
    parents_indices = np.random.choice(len(arrays), size=2, p=probability_distribution, replace=False)
    return arrays[parents_indices[0]], arrays[parents_indices[1]]

# 交叉関数（一点交叉）
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, parent1.shape[1])  # 交叉点をランダムに選択
    print(crossover_point)
    child1 = np.hstack((parent1[:, :crossover_point], parent2[:, crossover_point:]))
    child2 = np.hstack((parent2[:, :crossover_point], parent1[:, crossover_point:]))
    return child1, child2

# 突然変異関数
def mutate(array, mutation_rate=0.01):
    mutation_mask = np.random.rand(1, array.shape[1]) < mutation_rate
    array[mutation_mask] = np.random.rand(np.sum(mutation_mask))
    return array

# 遺伝的アルゴリズムの実行
def genetic_algorithm(arrays, scores, mutation_rate=0.05):
    # 親の選択
    parent1, parent2 = roulette_select(arrays, scores)
    
    # 交叉
    child1, child2 = crossover(parent1, parent2)
    
    # # 突然変異
    # child1 = mutate(child1, mutation_rate)
    # child2 = mutate(child2, mutation_rate)
    
    return child1, child2

# 以下繰り返す

In [ ]:
# フォルダ内のMIDIを再生
play_folder(midi_dir)

ここで満足いけば終了、だめなら以下を実行

In [ ]:
# UNDXバージョン、スコア入力

import glob
import numpy as np
import os

npy_paths = glob.glob(os.path.join(vectors_dir, '*.npy'))
midi_paths = glob.glob(os.path.join(midi_dir, '*.mid'))
npy_arrays = [np.load(file) for file in npy_paths]

# ユーザがMIDIを評価する
scores = np.array(list(
    map(int, input().split())))
print(scores)

# アルゴリズムの実行
# 最高評価の個体だけエリート選択で残すのでその分は空けておく
children = []
for i in range(NUM_OUTPUTS - 1):
    children.append(evolutionary_computation(npy_arrays, scores))


In [ ]:
# 一点交叉、スコアinput

import glob
import numpy as np
import os

# ユーザがMIDIを評価する
scores = np.array(list(
    map(int, input().split())))
print(scores)

npy_paths = glob.glob(os.path.join(vectors_dir, '*.npy'))
arrays = [np.load(file) for file in npy_paths]

# scoresの中で最大値のインデックスを取得、対応するarraysの要素をchild1に代入
child1 = arrays[np.argmax(scores)]
child2 = mutate(child1)
child3, child4 = genetic_algorithm(arrays, scores)

In [ ]:
# UNDXバージョン、親は手動選択

import glob
import numpy as np
import os

parent1_num = 1
parent2_num = 2
parent3_num = 3

npy_paths = glob.glob(os.path.join(vectors_dir, '*.npy'))
arrays = [np.load(file) for file in npy_paths]

# # アルゴリズムの実行
# # chilld1だけエリート選択で残す
child1 = arrays[parent1_num]
# child1 = undx(arrays[parent1_num],
#               arrays[parent2_num],
#               arrays[parent3_num])
child2 = undx(arrays[parent1_num],
              arrays[parent2_num],
              arrays[parent3_num])
child3 = undx(arrays[parent1_num],
              arrays[parent2_num],
              arrays[parent3_num])
child4 = undx(arrays[parent1_num],
              arrays[parent2_num],
              arrays[parent3_num])
child5 = undx(arrays[parent1_num],
              arrays[parent2_num],
              arrays[parent3_num])

In [ ]:
# 一点交叉input

# import glob

# parent1_num, parent2_num = map(int, input().split())

# npy_files = glob.glob(os.path.join(vectors_dir, '*.npy'))
# arrays = [np.load(file) for file in npy_files]

# child1 = arrays[parent1_num]
# child2, child3 = crossover(arrays[parent1_num], arrays[parent2_num])

In [ ]:
# 一点交叉手動

# import glob

# # ユーザがMIDIを評価する
# scores = np.array([2, 2, 2, 5])

# parent1_num = 0
# parent2_num = 1

# npy_files = glob.glob(os.path.join(vectors_dir, '*.npy'))
# arrays = [np.load(file) for file in npy_files]

# # アルゴリズムの実行
# child1, child2 = genetic_algorithm(arrays, scores)
# child3, child4 = genetic_algorithm(arrays, scores)

In [ ]:
# 世代を進める
generation += 1
midi_dir = os.path.join(OUTPUT_DIR, iec_dir, f"gen_{generation:03d}")
vectors_dir = os.path.join(OUTPUT_DIR, iec_dir, f"gen_{generation:03d}")
os.makedirs(vectors_dir)

for i in range(len(children)):
    np.save(os.path.join(vectors_dir, f'{i+1:02d}.npy'), children[i])

# np.save(vectors_dir+'\\01.npy', child1)
# np.save(vectors_dir+'\\02.npy', child2)
# np.save(vectors_dir+'\\03.npy', child3)
# np.save(vectors_dir+'\\04.npy', child4)
# np.save(vectors_dir+'\\05.npy', child5)

print(f"第{generation}世代に入りました")

In [ ]:
# 世代を進める
# generation += 1
# midi_dir = os.path.join(output_dir, iec_dir, f"midi\gen_{generation:03d}")
# vectors_dir = os.path.join(output_dir, iec_dir, f"vector\gen_{generation:03d}")
# os.makedirs(vectors_dir)

In [ ]:
# スクリプトに渡したい引数のリスト
args = [
    '--config='+MODEL_CONFIG,
    '--checkpoint_file='+CHECKPOINT_FILE,
    '--mode=vectors',
    '--vectors_dir='+vectors_dir,
    '--output_dir='+midi_dir,
    '--temperature='+str(TEMPERATURE)
]

# subprocess.runを使用してスクリプトを実行
# 引数はリストの形で渡す
result = subprocess.run(['python', SCRIPT_PATH] + args, cwd=os.getcwd(), capture_output=True, text=True)

# 実行結果の出力
# print('stdout:', result.stdout)
# print('stderr:', result.stderr)

In [ ]:
# エリート選択で保存

import shutil

npy_src = npy_paths[np.argmax(scores)]
midi_src = midi_paths[np.argmax(scores)]
# npy_dst = os.path.join(vectors_dir, '00.npy')
# midi_dst = os.path.join(midi_dir, '00.mid')
# shutil.copyfile(npy_src, npy_dst)
# shutil.copyfile(midi_src, midi_dst)
shutil.copy(npy_src, midi_dir)
shutil.copy(midi_src, midi_dir)